In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
#load data
from tensorflow.keras.datasets import fashion_mnist
(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

#Split data
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train_set, y_train_set, random_state=1)

#Preprocessing
x_train = x_train/ 255.0
x_valid = x_valid/ 255.0
x_test = x_test/ 255.0

In [5]:
def split_data(x, y):
    ind_B = (y==4) | (y==5)
    y_B = (y[ind_B]==5).astype(np.float32)
    y_A = y[~ind_B]
    y_A[y_A>5] -= 2
    return (x[~ind_B], y_A), (x[ind_B], y_B)
(x_train_A, y_train_A), (x_train_B, y_train_B) = split_data(x_train, y_train)
(x_valid_A, y_valid_A), (x_valid_B, y_valid_B) = split_data(x_valid, y_valid)
(x_test_A, y_test_A), (x_test_B, y_test_B) = split_data(x_test, y_test)

x_train_B, y_train_B = x_train_B[:100], y_train_B[:100]

In [7]:
x_train_A.shape, y_train_A[:10]

((35968, 28, 28), array([0, 7, 6, 2, 2, 1, 1, 1, 6, 7], dtype=uint8))

In [8]:
x_train_B.shape, y_train_B[:10]

((100, 28, 28), array([1., 0., 0., 0., 1., 1., 1., 0., 0., 0.], dtype=float32))

In [10]:
#先設置一個ModelA來對DataA進行訓練

keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

In [11]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for hidden_i in (200, 150, 100, 50):
    model_A.add(keras.layers.Dense(hidden_i, activation='relu'))
model_A.add(keras.layers.Dense(8,activation='softmax'))

In [12]:
model_A.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 200)               157000    
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 dense_2 (Dense)             (None, 100)               15100     
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 8)                 408       
                                                                 
Total params: 207,708
Trainable params: 207,708
Non-trai

In [13]:
model_A.compile(loss='sparse_categorical_crossentropy',
               optimizer=keras.optimizers.SGD(learning_rate=1e-3),
               metrics=['accuracy'])

In [14]:
train = model_A.fit(x_train_A, y_train_A, epochs=20,
                   validation_data=(x_valid_A, y_valid_A))

Epoch 1/20
1124/1124 [==============================] - 8s 6ms/step - loss: 1.5908 - accuracy: 0.6152 - val_loss: 1.0464 - val_accuracy: 0.7776
Epoch 2/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.7812 - accuracy: 0.8022 - val_loss: 0.6255 - val_accuracy: 0.8090
Epoch 3/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.5711 - accuracy: 0.8166 - val_loss: 0.5318 - val_accuracy: 0.8245
Epoch 4/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.5090 - accuracy: 0.8270 - val_loss: 0.4893 - val_accuracy: 0.8329
Epoch 5/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.4760 - accuracy: 0.8346 - val_loss: 0.4702 - val_accuracy: 0.8297
Epoch 6/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.4533 - accuracy: 0.8420 - val_loss: 0.4448 - val_accuracy: 0.8453
Epoch 7/20
1124/1124 [==============================] - 7s 6ms/step - loss: 0.4360 - accuracy: 0.8479 - val_loss: 0.4321 - val_accuracy:

In [15]:
model_A.evaluate(x_test_A, y_test_A)

250/250 [==============================] - 1s 3ms/step - loss: 0.3774 - accuracy: 0.8666


[0.37736156582832336, 0.8666250109672546]

In [16]:
model_A.save('model_A.h5')

In [19]:
#Model_B

model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for hidden_i in (300, 150, 100, 80):
    model_B.add(keras.layers.Dense(hidden_i, activation='relu'))
model_B.add(keras.layers.Dense(1, activation='sigmoid'))

In [20]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 300)               235500    
                                                                 
 dense_11 (Dense)            (None, 150)               45150     
                                                                 
 dense_12 (Dense)            (None, 100)               15100     
                                                                 
 dense_13 (Dense)            (None, 80)                8080      
                                                                 
 dense_14 (Dense)            (None, 1)                 81        
                                                                 
Total params: 303,911
Trainable params: 303,911
Non-tr

In [21]:
model_B.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

In [22]:
train = model_B.fit(x_train_B, y_train_B, epochs=20,
                   validation_data=(x_valid_B, y_valid_B))

Epoch 1/20
4/4 [==============================] - 2s 251ms/step - loss: 0.7131 - accuracy: 0.3900 - val_loss: 0.7092 - val_accuracy: 0.3757
Epoch 2/20
4/4 [==============================] - 0s 126ms/step - loss: 0.7044 - accuracy: 0.4000 - val_loss: 0.7007 - val_accuracy: 0.4505
Epoch 3/20
4/4 [==============================] - 0s 147ms/step - loss: 0.6955 - accuracy: 0.5200 - val_loss: 0.6894 - val_accuracy: 0.5421
Epoch 4/20
4/4 [==============================] - 0s 107ms/step - loss: 0.6838 - accuracy: 0.6200 - val_loss: 0.6774 - val_accuracy: 0.5984
Epoch 5/20
4/4 [==============================] - 0s 116ms/step - loss: 0.6722 - accuracy: 0.6800 - val_loss: 0.6698 - val_accuracy: 0.6216
Epoch 6/20
4/4 [==============================] - 0s 148ms/step - loss: 0.6643 - accuracy: 0.6800 - val_loss: 0.6622 - val_accuracy: 0.6344
Epoch 7/20
4/4 [==============================] - 0s 126ms/step - loss: 0.6566 - accuracy: 0.6800 - val_loss: 0.6538 - val_accuracy: 0.6435
Epoch 8/20
4/4 [====

In [23]:
model_B.evaluate(x_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.5799 - accuracy: 0.6435


[0.5798544883728027, 0.6434999704360962]

In [24]:
#Transfer Learning

#1. Load model_A
model_A = keras.models.load_model('model_A.h5')

In [25]:
#2. 使用model_A較低層

model_tr = keras.models.Sequential(model_A.layers[:-2])

In [26]:
model_tr.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 200)               157000    
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 dense_2 (Dense)             (None, 100)               15100     
                                                                 
Total params: 202,250
Trainable params: 202,250
Non-trainable params: 0
_________________________________________________________________


In [27]:
#3. 替換Model_A較高層

model_tr.add(keras.layers.Dense(80, activation='relu'))
model_tr.add(keras.layers.Dense(1, activation='sigmoid'))

In [28]:
model_tr.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 200)               157000    
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 dense_2 (Dense)             (None, 100)               15100     
                                                                 
 dense_15 (Dense)            (None, 80)                8080      
                                                                 
 dense_16 (Dense)            (None, 1)                 81        
                                                                 
Total params: 210,411
Trainable params: 210,411
Non-tr

In [29]:
#4.初期訓練(Epoch1~4)

for layer in model_tr.layers[:-2]:
    layer.trainable = False

In [30]:
model_tr.summary(show_trainable=True)

Model: "sequential_3"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 flatten (Flatten)           (None, 784)               0         N          
                                                                            
 dense (Dense)               (None, 200)               157000    N          
                                                                            
 dense_1 (Dense)             (None, 150)               30150     N          
                                                                            
 dense_2 (Dense)             (None, 100)               15100     N          
                                                                            
 dense_15 (Dense)            (None, 80)                8080      Y          
                                                                            
 dense_16 (Dense)            (None, 1)                

In [36]:
model_tr.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

In [37]:
train = model_tr.fit(x_train_B, y_train_B, epochs=4,
                    validation_data=(x_valid_B, y_valid_B))

Epoch 1/4
4/4 [==============================] - 1s 217ms/step - loss: 1.0867 - accuracy: 0.1000 - val_loss: 1.0018 - val_accuracy: 0.1253
Epoch 2/4
4/4 [==============================] - 0s 125ms/step - loss: 0.9868 - accuracy: 0.1800 - val_loss: 0.9166 - val_accuracy: 0.2426
Epoch 3/4
4/4 [==============================] - 0s 142ms/step - loss: 0.9072 - accuracy: 0.3000 - val_loss: 0.8355 - val_accuracy: 0.3999
Epoch 4/4
4/4 [==============================] - 0s 106ms/step - loss: 0.8318 - accuracy: 0.3900 - val_loss: 0.7879 - val_accuracy: 0.4569


In [38]:
#5.後期訓練(epochs 5~20)

for layer in model_tr.layers[:-2]:
    layer.trainable=True

In [39]:
model_tr.summary(show_trainable=True)

Model: "sequential_3"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 flatten (Flatten)           (None, 784)               0         Y          
                                                                            
 dense (Dense)               (None, 200)               157000    Y          
                                                                            
 dense_1 (Dense)             (None, 150)               30150     Y          
                                                                            
 dense_2 (Dense)             (None, 100)               15100     Y          
                                                                            
 dense_15 (Dense)            (None, 80)                8080      Y          
                                                                            
 dense_16 (Dense)            (None, 1)                

In [40]:
model_tr.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

In [41]:
train = model_tr.fit(x_train_B, y_train_B, epochs=16,
                    validation_data=(x_valid_B, y_valid_B))

Epoch 1/16
4/4 [==============================] - 2s 202ms/step - loss: 0.7792 - accuracy: 0.4600 - val_loss: 0.7321 - val_accuracy: 0.4700
Epoch 2/16
4/4 [==============================] - 0s 122ms/step - loss: 0.7215 - accuracy: 0.4800 - val_loss: 0.6807 - val_accuracy: 0.4842
Epoch 3/16
4/4 [==============================] - 0s 139ms/step - loss: 0.6700 - accuracy: 0.4900 - val_loss: 0.6325 - val_accuracy: 0.4963
Epoch 4/16
4/4 [==============================] - 0s 112ms/step - loss: 0.6215 - accuracy: 0.5000 - val_loss: 0.5975 - val_accuracy: 0.5057
Epoch 5/16
4/4 [==============================] - 0s 118ms/step - loss: 0.5883 - accuracy: 0.5000 - val_loss: 0.5626 - val_accuracy: 0.5347
Epoch 6/16
4/4 [==============================] - 0s 132ms/step - loss: 0.5504 - accuracy: 0.5400 - val_loss: 0.5288 - val_accuracy: 0.5873
Epoch 7/16
4/4 [==============================] - 0s 122ms/step - loss: 0.5165 - accuracy: 0.6300 - val_loss: 0.5009 - val_accuracy: 0.6418
Epoch 8/16
4/4 [====

In [42]:
model_tr.evaluate(x_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.3365 - accuracy: 0.9185


[0.33649739623069763, 0.9185000061988831]

In [ ]:
model_B.evaluate(x_testB)